<a href="https://colab.research.google.com/github/tubagokhan/GrinCH/blob/main/GraphInClusterArxivTextRank_nli_distilroberta_base_v2_SentenceDistrubtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install matplotlib
!pip install sentence_transformers
!pip install py-rouge==1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=40baedd5e62213abedc6b90deeae17e8752107c8881

In [3]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize,word_tokenize

from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import warnings
warnings.filterwarnings("ignore")
import re

from datasets import load_dataset
from scipy.spatial import distance

import math
from math import*

import rouge

import networkx as nx

import time
from transformers import logging
logging.set_verbosity_error()

import json
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# Preprocessing method
def preprocess_corpus(text):
    # Remove special characters and extra whitespaces
    text = re.sub(r"[^a-zA-Z0-9\s.]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

def textSentenceCount(Text):
    number_of_sentences = sent_tokenize(Text)
    count=(len(number_of_sentences))
    return count

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def createSummaryUsingKMeans(corpus, modelName):
    sentences = sent_tokenize(corpus)
    model = SentenceTransformer(modelName)
    sentence_embeddings = model.encode(sentences)

    optimum_clusters = find_optimum_clusters(sentence_embeddings, int(len(sentences) / 3))
    print("Optimum cluster number:", optimum_clusters)

    # Perform kmean clustering
    kmeans = KMeans(n_clusters=optimum_clusters, random_state=0, n_init='auto').fit(sentence_embeddings)

    chosen_sentence_indexes=[]
    cluster_rank=[]
    for cluster_id in range(optimum_clusters):
        cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
        cluster_weight=clusterWeight(sentence_embeddings, cluster_indices)
        cluster_rank.append(cluster_weight)
        chosen_sentence_index=text_rank(sentence_embeddings, cluster_indices)
        chosen_sentence_indexes.append(chosen_sentence_index)

    if optimum_clusters>9:
      choosen_clusters=k_highest_indices(cluster_rank, 9)
      chosen_sentence_indexes = [chosen_sentence_indexes[i] for i in choosen_clusters]

    sorted_indexes=sorted(chosen_sentence_indexes)
    saveSummariesinJsonFile(len(sentences), sorted_indexes,file_Path_Sent_Dist)

    chosen_sentences = []
    for chosen_sentence_index in sorted_indexes:
        chosen_sentences.append(sentences[chosen_sentence_index])

    summary = " ".join(chosen_sentences)

    return summary

def convert_int64_to_int(obj):
    if isinstance(obj, int):
        return int(obj)
    raise TypeError

def saveSummariesinJsonFile(num_sents, sorted_indexes,file_Path_Sent_Dist):
    filePath=file_Path_Sent_Dist
    # Create a dictionary with the required data structure
    data = {
        "num_sents": num_sents,
        "summary": [[int(index)] for index in sorted_indexes]
    }

    # Check if the file path exists
    if os.path.exists(filePath):
        try:
            # If the file exists and is not empty, read the existing data
            with open(filePath, "r") as file:
                existing_data = json.load(file)
        except json.JSONDecodeError:
            # If the file exists but contains invalid JSON data, initialize with an empty list
            existing_data = []
    else:
        # If the file does not exist, create a new file with the data
        existing_data = []

    # Append the new data to the existing data
    existing_data.append(data)

    # Write the updated data to the file
    with open(filePath, "w") as file:
        json.dump(existing_data, file, indent=2, default=convert_int64_to_int)

#function calculates the optimal number of clusters using the elbow method. The function plots the elbow curve, which shows the inertia values for different cluster numbers. The user can visually inspect the plot to determine the elbow point, indicating the optimal number of clusters.
def find_optimum_clusters(data, max_clusters):
    inertias = []
    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=0).fit(data)
        inertias.append(kmeans.inertia_)

    # Plotting the elbow curve
    #plt.plot(range(1, max_clusters + 1), inertias)
    #plt.xlabel("Number of Clusters")
    #plt.ylabel("Inertia")
    #plt.title("Elbow Curve")
    #plt.show()

    # Calculate the optimal number of clusters using the elbow method
    diff = np.diff(inertias)
    acceleration = np.diff(diff)
    opt_cluster_num = acceleration.argmin() + 2  # Adding 2 to get the index of the minimum acceleration
    return opt_cluster_num

def clusterWeight(sentence_embeddings, cluster_indices):
  length=len(cluster_indices)
  sentenceGraph =np.zeros((length, length))
  for x in range(length):
    for y in range(length):
      if x>y:
        similarity= cosine(sentence_embeddings[cluster_indices[x]],sentence_embeddings[cluster_indices[y]])
        sentenceGraph[x][y]=abs(similarity)
    #print(sentenceGraph)
  SumElement=(np.concatenate(sentenceGraph).sum())
  return round(SumElement,2)

def k_highest_indices(cluster_weight_list, k):
    # Enumerate the list to keep track of original indices
    enumerated_list = list(enumerate(cluster_weight_list))

    # Sort the enumerated list in descending order based on the float values
    sorted_list = sorted(enumerated_list, key=lambda x: x[1], reverse=True)

    # Extract the indices of the k highest elements
    k_highest_indices = [item[0] for item in sorted_list[:k]]

    return k_highest_indices

def text_rank(sentence_embeddings, indexes):
    # Filter embeddings based on the input indexes
    filtered_embeddings = np.array([sentence_embeddings[i] for i in indexes])

    # Calculate cosine similarity between filtered sentence embeddings
    similarity_matrix = cosine_similarity(filtered_embeddings, filtered_embeddings)

    # Create a graph using similarity matrix
    graph = nx.from_numpy_array(similarity_matrix)
    try:
      # Apply TextRank algorithm to rank the sentences
      scores = nx.pagerank(graph, max_iter=1000)
          # Get the index of the highest-ranked sentence
      highest_ranked_index = max(scores, key=scores.get)

      # Get the original index from the list of indexes
      highest_ranked_original_index = indexes[highest_ranked_index]
      return highest_ranked_original_index
    except:
      print("######### Error with PageRank")
      return indexes[0]





In [5]:
def save_data_to_json(file_path, summary, goldstandard, document):
    try:
        data = {
            "all_summary": [],
            "all_goldstandard": [],
            "all_document": []
        }

        try:
            with open(file_path, 'r') as file:
                data = json.load(file)
        except FileNotFoundError:
            pass

        all_summary = data.get('all_summary', [])
        all_goldstandard = data.get('all_goldstandard', [])
        all_document = data.get('all_document', [])

        all_summary.append(summary)
        all_goldstandard.append(goldstandard)
        all_document.append(document)

        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file)

        print("Data saved successfully.")
    except Exception as e:
        print(f"Error: {e}")

In [6]:
#modelName = 'all-mpnet-base-v2'
modelName ='nli-distilroberta-base-v2'
dataset = load_dataset("scientific_papers", "arxiv")
datasetName='-Arxiv'

path='/content/drive/Othercomputers/My Laptop/DriveAccess/'

# Define the file path
file_path = path+"GrinchAllGoldAndMySummaries-"+modelName+datasetName+".json"  # My Summaries, Gold Summaires, Documents
file_Path_Sent_Dist=path+"GrinchSentenceDistrubtionsOfSummaries-"+modelName+datasetName+".json"  # Sentence Distrubition Summaries, Summary Sentence Indexes


startTimeforOverall = time.time()
# document size 6440
N = 280   # HOW MANY DOCUMENT MALCOLLLMMMMMMM
startN = 1620 # Which document we will start

for d in range(N):
    startTimeforDocument = time.time()
    print("Document:", startN + d + 1)
    corpus = dataset['test']['article'][startN + d]
    corpus = preprocess_corpus(corpus)
    print("Document sentence number:", textSentenceCount(corpus))

    summary=""
    if textSentenceCount(corpus) > 8:
        summary = createSummaryUsingKMeans(corpus, modelName)
        print("Summary sentence number:", textSentenceCount(summary))
    else:
      summary=corpus
      print("Corpus is less than 9 sentence, summarization didn't apply sentence number:", textSentenceCount(corpus))


    save_data_to_json(file_path, summary, dataset['test']['abstract'][startN + d], dataset['test']['article'][startN + d])

    elapsedTimeforDocument = time.time() - startTimeforDocument
    elapsedTimeforAll = time.time() - startTimeforOverall
    print('Document processing time: '+time.strftime("%M:%S", time.gmtime(elapsedTimeforDocument)))
    print('Total processing time: '+time.strftime("%d:%H:%M:%S", time.gmtime(elapsedTimeforAll)))

    print("----------------------------------")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

Document: 1621
Document sentence number: 346


Optimum cluster number: 46
Summary sentence number: 9
Data saved successfully.
Document processing time: 02:26
Total processing time: 01:00:02:26
----------------------------------
Document: 1622
Document sentence number: 328
Optimum cluster number: 38
Summary sentence number: 9
Data saved successfully.
Document processing time: 02:02
Total processing time: 01:00:04:28
----------------------------------
Document: 1623
Document sentence number: 126
Optimum cluster number: 9
Summary sentence number: 9
Data saved successfully.
Document processing time: 00:16
Total processing time: 01:00:04:45
----------------------------------
Document: 1624
Document sentence number: 293
Optimum cluster number: 7
Summary sentence number: 7
Data saved successfully.
Document processing time: 01:32
Total processing time: 01:00:06:18
----------------------------------
Document: 1625
Document sentence number: 230
Optimum cluster number: 31
Summary sentence number: 9
Data saved successfully.
Document processing

In [7]:
def prepare_results(m, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(m, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

def rougeEvaluation(all_hypothesis, all_references):

    for aggregator in ['Avg']:
        print('Evaluation with {}'.format(aggregator))
        apply_avg = aggregator == 'Avg'
        apply_best = aggregator == 'Best'

        evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                               max_n=4,
                               limit_length=False,
                               length_limit=1000,
                               length_limit_type='words',
                               apply_avg=apply_avg,
                               apply_best=apply_best,
                               alpha=0.2, # Default F1_score
                               weight_factor=1.2,
                               stemming=True)

        scores = evaluator.get_scores(all_hypothesis, all_references)

        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
                for hypothesis_id, results_per_ref in enumerate(results):
                    nb_references = len(results_per_ref['p'])
                    for reference_id in range(nb_references):
                        print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                        print('\t' + prepare_results(metric,results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
                print()
            else:
                print(prepare_results(metric, results['p'], results['r'], results['f']))
        print()

In [8]:
#file_path="/content/drive/Othercomputers/My Laptop/DriveAccess/GrinchAllGoldAndMySummaries-nli-distilroberta-base-v2-PubMed.json"

In [9]:
try:
  with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    mysummaries = data.get('all_summary', [])
    goldstandardsummaries = data.get('all_goldstandard', [])
    documents = data.get('all_document', [])

    print("Data read successfully.")
except FileNotFoundError:
  print(f"Error: File '{file_path}' not found.")
  mysummaries, goldstandardsummaries, documents  = []
except Exception as e:
  print(f"Error: {e}")
  mysummaries, goldstandardsummaries, documents  = []

Data read successfully.


In [10]:
len(documents)

1900

In [11]:
dataset = load_dataset("scientific_papers", "arxiv")
len(dataset['test']['article'])

6440

In [12]:
rougeEvaluation(mysummaries, goldstandardsummaries)

Evaluation with Avg
	rouge-1:	P: 29.38	R: 51.19	F1: 42.77
	rouge-2:	P:  8.58	R: 15.07	F1: 12.54
	rouge-3:	P:  3.17	R:  5.54	F1:  4.61
	rouge-4:	P:  1.54	R:  2.67	F1:  2.23
	rouge-l:	P: 28.05	R: 44.51	F1: 38.72
	rouge-w:	P: 12.93	R: 11.61	F1: 11.46



In [13]:
def find_duplicates_indexes(lst):
    duplicates = {}
    for i, item in enumerate(lst):
        if item in duplicates:
            duplicates[item].append(i)
        else:
            duplicates[item] = [i]

    # Extract index lists for duplicate values
    duplicate_indexes = [indexes for indexes in duplicates.values() if len(indexes) > 1]

    # Flatten the list of index lists
    flattened_indexes = [index for indexes in duplicate_indexes for index in indexes]

    return flattened_indexes



result = find_duplicates_indexes(goldstandardsummaries)
print(result)

result = find_duplicates_indexes(documents)
print(result)

[]
[]


In [14]:
def find_missing_values_indexes(list1, list2):
    missing_indexes = [index for index, value in enumerate(list1) if value not in list2]
    return missing_indexes

# Example usage:
list1 = dataset['test']['article']
list2 = documents
missing_indexes = find_missing_values_indexes(list1, list2)
print(missing_indexes)
len(missing_indexes)

[1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 206

4540

In [15]:
sentencenumber=0
for i in range(len(mysummaries)):
  sentencenumber=sentencenumber+textSentenceCount(mysummaries[i])
avg=sentencenumber/len(mysummaries)
avg

8.776842105263158

In [16]:
sentencenumber=0
for i in range(len(goldstandardsummaries)):
  sentencenumber=sentencenumber+textSentenceCount(goldstandardsummaries[i])
avg=sentencenumber/len(goldstandardsummaries)
avg

6.262105263157895

In [17]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [18]:
print(len(mysummaries))

1900
